# Nearest Neighbors

In [1]:
import graphlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from __future__ import division
import math
import time

In [2]:
wiki = graphlab.SFrame("E:\\Machine Learning\\U.W\\Cluster and Retrieval\\people_wiki.gl/")
wiki.save("E:\\Machine Learning\\U.W\\Cluster and Retrieval\\people.wiki.csv", format="csv")
wiki = pd.read_csv("E:\\Machine Learning\\U.W\\Cluster and Retrieval\\people.wiki.csv")

This non-commercial license of GraphLab Create for academic use is assigned to lxn1021@gmail.com and will expire on November 18, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Xiaoning\AppData\Local\Temp\graphlab_server_1560060853.log.0


In [3]:
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## Extract word count vectors

In [4]:
def word_count(text):
    words = {}
    
    words_list = text.split()
    for i in range(len(words_list)):
        if words_list[i] in words:
            words[words_list[i]] += 1
        else:
            words[words_list[i]] = 1
    
    
    return words

In [5]:
wiki["word_count"] = wiki["text"].apply(word_count)

In [6]:
wiki.head()

,URI,name,text,word_count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{u'selection': 1, u'carltons': 1, u'being': 1,..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{u'precise': 1, u'thomas': 1, u'they': 1, u'di..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{u'just': 1, u'issued': 1, u'mainly': 1, u'nom..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{u'all': 1, u'bauforschung': 1, u'just': 1, u'..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{u'legendary': 1, u'gangstergenka': 1, u'legen..."


## Find nearest neighbors

In [7]:
def words_all(text):
    
    for word in text.keys():
        if word not in words_list:
            words_list.append(word)
            
            
    return len(words_list)

In [8]:
sample = wiki.head(1000)

In [9]:
sample["id"] = sample.index

c:\python27\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
sample_new = sample["word_count"].apply(pd.Series).stack()
sample_new = sample_new.reset_index()
sample_new.columns = ["id", "feature", "value"]

In [11]:
sample_new

,id,feature,value
0,0,10,1.0
1,0,1979,1.0
2,0,19982000,1.0
3,0,2000,1.0
4,0,2001,1.0
5,0,2002,1.0
6,0,2003,1.0
7,0,2005,1.0
8,0,2006,1.0
9,0,2007,1.0


In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
f = LabelEncoder()
sample_new_label = f.fit_transform(sample_new["feature"])

sample_new["feature_id"] = sample_new_label

In [14]:
sample_new

,id,feature,value,feature_id
0,0,10,1.0,19
1,0,1979,1.0,496
2,0,19982000,1.0,728
3,0,2000,1.0,768
4,0,2001,1.0,799
5,0,2002,1.0,817
6,0,2003,1.0,835
7,0,2005,1.0,883
8,0,2006,1.0,903
9,0,2007,1.0,922


In [15]:
from scipy.sparse import csr_matrix

v = np.array(sample_new["value"])
i = np.array(sample_new["id"])
j = np.array(sample_new["feature_id"])

row = sample_new["id"].max() + 1
col = j.max() + 1

mat = csr_matrix((v, (i, j)), shape=(row, col))
features_toarray = mat.toarray()

In [16]:
features_toarray.shape

(1000L, 34257L)

In [18]:
from sklearn.neighbors import NearestNeighbors

In [19]:
model = NearestNeighbors(n_neighbors=10, algorithm="brute").fit(features_toarray)

In [20]:
model

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [21]:
model.kneighbors(features_toarray)

(array([[ 0.        , 21.9089023 , 22.11334439, ..., 24.39262184,
         24.71841419, 24.85960579],
        [ 0.        , 22.38302929, 22.42766149, ..., 23.57965225,
         23.62202362, 23.8117618 ],
        [ 0.        , 22.20360331, 22.4053565 , ..., 22.737634  ,
         22.737634  , 22.89104628],
        ...,
        [ 0.        , 26.34387974, 27.18455444, ..., 28.39013913,
         28.53068524, 28.58321186],
        [ 0.        , 29.25747768, 31.51190251, ..., 32.06243908,
         32.10918872, 32.17141588],
        [ 0.        , 24.14539294, 24.73863375, ..., 25.37715508,
         25.3968502 , 25.53429067]]),
 array([[  0, 621, 314, ..., 648, 308, 174],
        [  1, 127,  67, ..., 717, 592, 540],
        [  2, 407, 127, ..., 600, 240, 756],
        ...,
        [997, 884, 479, ..., 352, 621, 308],
        [998,  16, 866, ...,  73, 461,  60],
        [999, 687, 691, ..., 230, 221, 561]], dtype=int64))

In [22]:
distance = model.kneighbors(features_toarray)[0]
neighbors = model.kneighbors(features_toarray)[1]

In [23]:
query_id = sample[sample["name"] == "Jack Paterson"]["id"].iloc[0]

query_neighbors = neighbors[query_id]
query_distance = distance[query_id]
query_name = sample[sample["id"]==query_id]["name"].iloc[0]
query_reference = sample.iloc[list(query_neighbors)]["name"]

In [24]:
query_table = pd.DataFrame({"query_label": query_name, "reference_label": query_reference, "distance": query_distance}).reset_index()
query_table = query_table.drop(["index"], axis=1)
query_table["rank"] = query_table.index + 1
query_table = query_table[["query_label", "reference_label", "distance", "rank"]]

In [25]:
query_table

,query_label,reference_label,distance,rank
0,Jack Paterson,Jack Paterson,0.000000,1
1,Jack Paterson,Glen Pine,23.685439,2
2,Jack Paterson,Andrew Bernstein,24.269322,3
3,Jack Paterson,David Shaughnessy,24.413111,4
4,Jack Paterson,Devyani Saltzman,24.433583,5
5,Jack Paterson,John Tarnoff,24.494897,6
6,Jack Paterson,Robert Katona,24.596748,7
7,Jack Paterson,Ronald Caltabiano,24.698178,8
8,Jack Paterson,Alan Moir,24.839485,9
9,Jack Paterson,Rajiva Wijesinha,24.859606,10


In [26]:
def top_words(name):
    row = sample[sample["name"] == name]
    word_count_table = row["word_count"].apply(pd.Series).stack()
    word_count_table = word_count_table.reset_index()
    word_count_table = word_count_table.drop(["level_0"], axis=1)
    word_count_table.columns = ["word", "count"]
    word_count_table = word_count_table.sort_values(by=["count"], ascending=False)
    
    
    return word_count_table

In [27]:
jack_words = top_words("Jack Paterson")

jack_words

,word,count
141,the,24
7,and,15
101,of,12
142,theatre,7
62,for,6
74,in,5
78,is,4
70,he,4
0,a,4
157,with,4


In [28]:
glen_words = top_words("Glen Pine")

glen_words

,word,count
113,the,20
11,and,10
75,of,7
77,on,5
51,he,5
84,pine,4
57,is,4
104,slackers,3
54,in,3
52,his,3


In [29]:
jack_words = jack_words.rename(columns = {"count": "count - jack"})
glen_words = glen_words.rename(columns = {"count": "count - glen"})

In [30]:
combined_words = jack_words.set_index("word").join(glen_words.set_index("word"))

combined_words = combined_words.reset_index()

In [31]:
common_words = set(combined_words.sort_values(by=["count - jack"], ascending=False)["word"][:3])

def has_top_words(word_count_vector):
    unique_words = set(word_count_vector.keys())
    
    return common_words.issubset(unique_words)


sample["has_top_words"] = sample["word_count"].apply(has_top_words)

sample["has_top_words"].sum()

c:\python27\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


994

## TF-IDF to the rescue

In [32]:
def tfidf(text):
    words_list = text.keys()
    tfidf = {}
    
    for i in range(len(text)):
        # Calculate TF
        word = words_list[i]
        tf = text[word]
        
        # Calculate IDF
        in_corpus = 0
        for j in range(len(sample)):
            if word in sample["word_count"][j].keys():
                in_corpus += 1
                
        idf = math.log(len(sample)/(1+in_corpus))
        
        # Calculate TF_IDF
        tfidf[word] = tf * idf
        
    
    return tfidf            

In [33]:
start = time.time()

sample["tf_idf"] = sample["word_count"].apply(tfidf)

time.time() - start

c:\python27\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


5314.601999998093

In [35]:
sample.head()

,URI,name,text,word_count,id,has_top_words,tf_idf
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{u'selection': 1, u'carltons': 1, u'being': 1,...",0,True,"{u'selection': 3.96331629982, u'carltons': 5.5..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{u'precise': 1, u'thomas': 1, u'they': 1, u'di...",1,True,"{u'precise': 5.52146091786, u'thomas': 3.21887..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{u'just': 1, u'issued': 1, u'mainly': 1, u'nom...",2,True,"{u'just': 2.60369018578, u'issued': 4.07454193..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{u'all': 1, u'bauforschung': 1, u'just': 1, u'...",3,True,"{u'englishreading': 6.21460809842, u'all': 1.6..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{u'legendary': 1, u'gangstergenka': 1, u'legen...",4,True,"{u'they': 1.70374859191, u'gangstergenka': 6.2..."


In [40]:
sample_new_tfidf = sample["tf_idf"].apply(pd.Series).stack()
sample_new_tfidf = sample_new_tfidf.reset_index()
sample_new_tfidf.columns = ["id", "feature", "value"]

In [41]:
sample_new_tfidf

,id,feature,value
0,0,10,2.302585
1,0,1979,2.603690
2,0,19982000,6.214608
3,0,2000,1.944911
4,0,2001,2.009915
5,0,2002,1.944911
6,0,2003,1.883875
7,0,2005,1.714798
8,0,2006,1.518684
9,0,2007,1.575036


In [42]:
from sklearn.preprocessing import LabelEncoder

In [43]:
f = LabelEncoder()
sample_new_tfidf_label = f.fit_transform(sample_new_tfidf["feature"])

sample_new_tfidf["feature_id"] = sample_new_tfidf_label

In [44]:
sample_new_tfidf

,id,feature,value,feature_id
0,0,10,2.302585,19
1,0,1979,2.603690,496
2,0,19982000,6.214608,728
3,0,2000,1.944911,768
4,0,2001,2.009915,799
5,0,2002,1.944911,817
6,0,2003,1.883875,835
7,0,2005,1.714798,883
8,0,2006,1.518684,903
9,0,2007,1.575036,922


In [45]:
v = np.array(sample_new_tfidf["value"])
i = np.array(sample_new_tfidf["id"])
j = np.array(sample_new_tfidf["feature_id"])

row = sample_new_tfidf["id"].max() + 1
col = j.max() + 1

mat = csr_matrix((v, (i, j)), shape=(row, col))
features_toarray = mat.toarray()

In [46]:
features_toarray.shape

(1000L, 34257L)

In [47]:
model_tfidf = NearestNeighbors(n_neighbors=10, algorithm="brute").fit(features_toarray)

In [48]:
model_tfidf

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [49]:
model.kneighbors(features_toarray)

(array([[57.83885877, 64.03090639, 64.52692039, ..., 66.28181179,
         66.52419011, 66.56511802],
        [62.86490407, 78.37758842, 78.46503006, ..., 78.93693602,
         78.93913436, 78.95452279],
        [51.33108211, 64.33225322, 64.42541307, ..., 65.52508717,
         65.54697355, 65.57340762],
        ...,
        [75.77050599, 84.60654473, 84.72308886, ..., 85.00720701,
         85.01988516, 85.07116574],
        [77.17422768, 86.91803828, 87.13875886, ..., 87.63533162,
         87.70398304, 87.76680227],
        [60.20850673, 73.87171353, 73.87911355, ..., 74.45257124,
         74.46446889, 74.49829742]]),
 array([[  0,  99, 118, ..., 770, 827, 419],
        [  1, 419, 534, ..., 628, 972, 957],
        [  2, 815, 419, ..., 561, 687, 760],
        ...,
        [997, 534, 419, ..., 747, 724,  79],
        [998, 972, 419, ..., 835,   1, 756],
        [999, 687,   4, ..., 500, 402, 740]], dtype=int64))

In [50]:
distance_tfidf = model.kneighbors(features_toarray)[0]
neighbors_tfidf = model.kneighbors(features_toarray)[1]

In [63]:
query_id = sample[sample["name"] == "Jack Paterson"]["id"].iloc[0]

query_neighbors_tfidf = neighbors_tfidf[query_id]
query_distance_tfidf = distance[query_id]
query_name_tfidf = sample[sample["id"]==query_id]["name"].iloc[0]
query_reference_tfidf = sample.iloc[list(query_neighbors_tfidf)]["name"]

In [73]:
query_table_tfidf = pd.DataFrame({"query_label": query_name_tfidf, \
                                  "reference_label": query_reference_tfidf, "distance": query_distance_tfidf}).reset_index()
query_table_tfidf = query_table_tfidf.drop(["index"], axis=1)
query_table_tfidf["rank"] = query_table_tfidf.index + 1
query_table_tfidf = query_table_tfidf[["query_label", "reference_label", "distance", "rank"]]

In [74]:
query_table_tfidf

,query_label,reference_label,distance,rank
0,Jack Paterson,Jack Paterson,0.000000,1
1,Jack Paterson,Harry Fraud,23.685439,2
2,Jack Paterson,Grant Gillespie (writer),24.269322,3
3,Jack Paterson,Bartosz Brenes,24.413111,4
4,Jack Paterson,Avi Muchnick,24.433583,5
5,Jack Paterson,Shawn Pelton,24.494897,6
6,Jack Paterson,Richard Allen Davis,24.596748,7
7,Jack Paterson,Cogie Domingo,24.698178,8
8,Jack Paterson,Larry Sawyer (poet),24.839485,9
9,Jack Paterson,Bo Hampton,24.859606,10


In [77]:
def top_words_tfidf(name):
    row = sample[sample["name"] == name]
    tfidf_table = row["tf_idf"].apply(pd.Series).stack()
    tfidf_table = tfidf_table.reset_index()
    tfidf_table = tfidf_table.drop(["level_0"], axis=1)
    tfidf_table.columns = ["word", "tf_idf"]
    tfidf_table = tfidf_table.sort_values(by=["tf_idf"], ascending=False)
    
    
    return tfidf_table

In [79]:
jack_tf_idf = top_words_tfidf("Jack Paterson")

jack_tf_idf

,word,tf_idf
142,theatre,20.562244
60,firehall,12.429216
23,bouchewhacked,12.429216
114,playwrights,11.618286
39,conspiracy,10.231992
133,shakespeare,9.421061
106,outstanding,9.370697
46,directing,8.399410
161,workshop,8.399410
43,credits,7.633426


In [80]:
harry_tf_idf = top_words_tfidf("Harry Fraud")

harry_tf_idf

,word,tf_idf
51,fraud,23.025851
102,rappers,12.429216
82,mixtape,11.618286
54,fulllength,10.231992
29,brooklyn,8.270333
131,titled,6.762790
119,solo,6.293110
22,av,6.214608
38,curreny,6.214608
31,chinx,6.214608


In [84]:
jack_tf_idf = jack_tf_idf.rename(columns={"tf_idf": "tf_idf_jack"})
harry_tf_idf = harry_tf_idf.rename(columns={"tf_idf": "tf_idf_harry"})

In [88]:
combined_words_tfidf = jack_tf_idf.set_index("word").join(harry_tf_idf.set_index("word"))

combined_words_tfidf = combined_words_tfidf.reset_index()

In [89]:
combined_words_tfidf

,word,tf_idf_jack,tf_idf_harry
0,theatre,20.562244,NaN
1,firehall,12.429216,NaN
2,bouchewhacked,12.429216,NaN
3,playwrights,11.618286,NaN
4,conspiracy,10.231992,NaN
5,shakespeare,9.421061,NaN
6,outstanding,9.370697,NaN
7,directing,8.399410,NaN
8,workshop,8.399410,NaN
9,credits,7.633426,NaN


In [95]:
common_words_tfidf = set(combined_words_tfidf.sort_values(by=["tf_idf_jack"], ascending=False)["word"][:3])

def has_top_words_tfidf(word_count_vector):
    unique_words = set(word_count_vector.keys())
    
    return common_words_tfidf.issubset(unique_words)


sample["has_top_words_tfidf"] = sample["tf_idf"].apply(has_top_words_tfidf)

sample["has_top_words_tfidf"].sum()

c:\python27\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1

## Choosing metrics

In [154]:
def compute_length(text):
    return len(text.split())
        

sample["length"] = sample["text"].apply(compute_length)

c:\python27\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [155]:
sample.head()

,URI,name,text,word_count,id,has_top_words,tf_idf,has_top_words_tfidf,length
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{u'selection': 1, u'carltons': 1, u'being': 1,...",0,True,"{u'selection': 3.96331629982, u'carltons': 5.5...",False,251
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{u'precise': 1, u'thomas': 1, u'they': 1, u'di...",1,True,"{u'precise': 5.52146091786, u'thomas': 3.21887...",False,223
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{u'just': 1, u'issued': 1, u'mainly': 1, u'nom...",2,True,"{u'just': 2.60369018578, u'issued': 4.07454193...",False,226
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{u'all': 1, u'bauforschung': 1, u'just': 1, u'...",3,True,"{u'englishreading': 6.21460809842, u'all': 1.6...",False,377
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{u'legendary': 1, u'gangstergenka': 1, u'legen...",4,True,"{u'they': 1.70374859191, u'gangstergenka': 6.2...",False,201


**Distance metric: euclidean**

In [158]:
model_tfidf_euclidean = NearestNeighbors(n_neighbors=10, algorithm="brute", metric="euclidean").fit(features_toarray)

In [159]:
distance_tfidf_euclidean = model_tfidf_euclidean.kneighbors(features_toarray)[0]
neighbors_tfidf_euclidean = model_tfidf_euclidean.kneighbors(features_toarray)[1]

In [193]:
query_id = sample[sample["name"] == "Jack Paterson"]["id"].iloc[0]

query_neighbors_tfidf_euclidean = neighbors_tfidf_euclidean[query_id]
query_distance_tfidf_euclidean = distance_tfidf_euclidean[query_id]
query_name_tfidf_euclidean = sample[sample["id"]==query_id]["name"].iloc[0]
query_reference_tfidf_euclidean = sample.iloc[list(query_neighbors_tfidf_euclidean)]["name"]
query_length_tfidf_euclidean = sample.iloc[list(query_neighbors_tfidf_euclidean)]["length"]

In [201]:
query_table_tfidf_euclidean = pd.DataFrame({"query_label": query_name_tfidf_euclidean, "reference_label": query_reference_tfidf_euclidean, \
                                            "distance": query_distance_tfidf_euclidean, "length": query_length_tfidf_euclidean}).reset_index()
query_table_tfidf_euclidean = query_table_tfidf_euclidean.drop(["index"], axis=1)
query_table_tfidf_euclidean["rank"] = query_table_tfidf_euclidean.index + 1
query_table_tfidf_euclidean = query_table_tfidf_euclidean[["query_label", "reference_label", "distance", "length", "rank"]]

In [202]:
query_table_tfidf_euclidean

,query_label,reference_label,distance,length,rank
0,Jack Paterson,Jack Paterson,0.000000,263,1
1,Jack Paterson,Vincenzo Visco,78.054579,204,2
2,Jack Paterson,Scott Sandage,78.602133,209,3
3,Jack Paterson,Ronnie Wallwork,79.919327,209,4
4,Jack Paterson,Alan Fletcher (composer),79.964910,211,5
5,Jack Paterson,Anna Friel,79.994334,240,6
6,Jack Paterson,Grant Gillespie (writer),80.071865,204,7
7,Jack Paterson,Shaka Hislop,80.160485,227,8
8,Jack Paterson,Anthony Whelan,80.476829,208,9
9,Jack Paterson,Mammootty,80.561048,211,10


**Distance metric: minkowski (default)**

In [203]:
model_tfidf_minkowski = NearestNeighbors(n_neighbors=10, algorithm="brute", metric="minkowski").fit(features_toarray)

In [204]:
distance_tfidf_minkowski = model_tfidf_minkowski.kneighbors(features_toarray)[0]
neighbors_tfidf_minkowski = model_tfidf_minkowski.kneighbors(features_toarray)[1]

In [213]:
query_id = sample[sample["name"] == "Jack Paterson"]["id"].iloc[0]

query_neighbors_tfidf_minkowski = neighbors_tfidf_minkowski[query_id]
query_distance_tfidf_minkowski = distance_tfidf_minkowski[query_id]
query_name_tfidf_minkowski = sample[sample["id"]==query_id]["name"].iloc[0]
query_reference_tfidf_minkowski = sample.iloc[list(query_neighbors_tfidf_minkowski)]["name"]
query_length_tfidf_minkowski = sample.iloc[list(query_neighbors_tfidf_minkowski)]["length"]

In [215]:
query_table_tfidf_minkowski = pd.DataFrame({"query_label": query_name_tfidf_minkowski, "reference_label": query_reference_tfidf_minkowski, \
                                            "distance": query_distance_tfidf_minkowski, "length": query_length_tfidf_minkowski}).reset_index()
query_table_tfidf_minkowski = query_table_tfidf_minkowski.drop(["index"], axis=1)
query_table_tfidf_minkowski["rank"] = query_table_tfidf_minkowski.index + 1
query_table_tfidf_minkowski = query_table_tfidf_minkowski[["query_label", "reference_label", "distance", "length", "rank"]]

In [216]:
query_table_tfidf_minkowski

,query_label,reference_label,distance,length,rank
0,Jack Paterson,Jack Paterson,0.000000,263,1
1,Jack Paterson,Vincenzo Visco,78.054579,204,2
2,Jack Paterson,Scott Sandage,78.602133,209,3
3,Jack Paterson,Ronnie Wallwork,79.919327,209,4
4,Jack Paterson,Alan Fletcher (composer),79.964910,211,5
5,Jack Paterson,Anna Friel,79.994334,240,6
6,Jack Paterson,Grant Gillespie (writer),80.071865,204,7
7,Jack Paterson,Shaka Hislop,80.160485,227,8
8,Jack Paterson,Anthony Whelan,80.476829,208,9
9,Jack Paterson,Mammootty,80.561048,211,10


**Distance metric: cosine**

In [217]:
model_tfidf_cosine = NearestNeighbors(n_neighbors=10, algorithm="brute", metric="cosine").fit(features_toarray)

In [219]:
distance_tfidf_cosine = model_tfidf_cosine.kneighbors(features_toarray)[0]
neighbors_tfidf_cosine = model_tfidf_cosine.kneighbors(features_toarray)[1]

In [233]:
query_id = sample[sample["name"] == "Jack Paterson"]["id"].iloc[0]

query_neighbors_tfidf_cosine = neighbors_tfidf_cosine[query_id]
query_distance_tfidf_cosine = distance_tfidf_cosine[query_id]
query_name_tfidf_cosine = sample[sample["id"]==query_id]["name"].iloc[0]
query_reference_tfidf_cosine = sample.iloc[list(query_neighbors_tfidf_cosine)]["name"]
query_length_tfidf_cosine = sample.iloc[list(query_neighbors_tfidf_cosine)]["length"]

In [240]:
query_table_tfidf_cosine = pd.DataFrame({"query_label": query_name_tfidf_cosine, "reference_label": query_reference_tfidf_cosine, \
                                            "distance": query_distance_tfidf_cosine, "length": query_length_tfidf_cosine}).reset_index()
query_table_tfidf_cosine = query_table_tfidf_cosine.drop(["index"], axis=1)
query_table_tfidf_cosine["rank"] = query_table_tfidf_cosine.index + 1
query_table_tfidf_cosine = query_table_tfidf_cosine[["query_label", "reference_label", "distance", "length", "rank"]]

In [241]:
query_table_tfidf_cosine

,query_label,reference_label,distance,length,rank
0,Jack Paterson,Jack Paterson,0.000000,263,1
1,Jack Paterson,David Eldridge (dramatist),0.818638,256,2
2,Jack Paterson,Rebecca Northan,0.846577,492,3
3,Jack Paterson,Stephen Evans (actor),0.893031,423,4
4,Jack Paterson,Martin Allen (writer),0.893244,398,5
5,Jack Paterson,Trevor Ferguson,0.895519,633,6
6,Jack Paterson,Rosalind Harris,0.896752,341,7
7,Jack Paterson,Belinda Wollaston,0.904185,230,8
8,Jack Paterson,Michael Rowe (director),0.907524,223,9
9,Jack Paterson,Lucien Bourjeily,0.911279,357,10
